In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/
%pwd
%rm -rf darknet
!git clone https://github.com/roboflow-ai/darknet.git
%cd /content/darknet/
%rm Makefile

In [ ]:
%%writefile Makefile
GPU=1
CUDNN=1
CUDNN_HALF=0
OPENCV=1
AVX=0
OPENMP=0
LIBSO=1
ZED_CAMERA=0
ZED_CAMERA_v2_8=0
USE_CPP=0
DEBUG=0

#Modificacion 11marzo
#-gencode arch=compute_30,code=sm_30 \
ARCH= -gencode arch=compute_35,code=sm_35 \
      -gencode arch=compute_50,code=[sm_50,compute_50] \
      -gencode arch=compute_52,code=[sm_52,compute_52] \
	    -gencode arch=compute_61,code=[sm_61,compute_61]

OS := $(shell uname)

VPATH=./src/
EXEC=darknet
OBJDIR=./obj/

ifeq ($(LIBSO), 1)
LIBNAMESO=libdarknet.so
APPNAMESO=uselib
endif

ifeq ($(USE_CPP), 1)
CC=g++
else
CC=gcc
endif

CPP=g++ -std=c++11
NVCC=nvcc
OPTS=-Ofast
LDFLAGS= -lm -pthread
COMMON= -Iinclude/ -I3rdparty/stb/include
CFLAGS=-Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC

ifeq ($(DEBUG), 1)
#OPTS= -O0 -g
#OPTS= -Og -g
COMMON+= -DDEBUG
CFLAGS+= -DDEBUG
else
ifeq ($(AVX), 1)
CFLAGS+= -ffp-contract=fast -mavx -mavx2 -msse3 -msse4.1 -msse4.2 -msse4a
endif
endif

CFLAGS+=$(OPTS)

ifneq (,$(findstring MSYS_NT,$(OS)))
LDFLAGS+=-lws2_32
endif

ifeq ($(OPENCV), 1)
COMMON+= -DOPENCV
CFLAGS+= -DOPENCV
LDFLAGS+= `pkg-config --libs opencv4 2> /dev/null || pkg-config --libs opencv`
COMMON+= `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv`
endif

ifeq ($(OPENMP), 1)
CFLAGS+= -fopenmp
LDFLAGS+= -lgomp
endif

ifeq ($(GPU), 1)
COMMON+= -DGPU -I/usr/local/cuda/include/
CFLAGS+= -DGPU
ifeq ($(OS),Darwin) #MAC
LDFLAGS+= -L/usr/local/cuda/lib -lcuda -lcudart -lcublas -lcurand
else
LDFLAGS+= -L/usr/local/cuda/lib64 -lcuda -lcudart -lcublas -lcurand
endif
endif

ifeq ($(CUDNN), 1)
COMMON+= -DCUDNN
ifeq ($(OS),Darwin) #MAC
CFLAGS+= -DCUDNN -I/usr/local/cuda/include
LDFLAGS+= -L/usr/local/cuda/lib -lcudnn
else
CFLAGS+= -DCUDNN -I/usr/local/cudnn/include
LDFLAGS+= -L/usr/local/cudnn/lib64 -lcudnn
endif
endif

ifeq ($(CUDNN_HALF), 1)
COMMON+= -DCUDNN_HALF
CFLAGS+= -DCUDNN_HALF
ARCH+= -gencode arch=compute_70,code=[sm_70,compute_70]
endif

ifeq ($(ZED_CAMERA), 1)
CFLAGS+= -DZED_STEREO -I/usr/local/zed/include
ifeq ($(ZED_CAMERA_v2_8), 1)
LDFLAGS+= -L/usr/local/zed/lib -lsl_core -lsl_input -lsl_zed
#-lstdc++ -D_GLIBCXX_USE_CXX11_ABI=0
else
LDFLAGS+= -L/usr/local/zed/lib -lsl_zed
#-lstdc++ -D_GLIBCXX_USE_CXX11_ABI=0
endif
endif

OBJ=image_opencv.o http_stream.o gemm.o utils.o dark_cuda.o convolutional_layer.o list.o image.o activations.o im2col.o col2im.o blas.o crop_layer.o dropout_layer.o maxpool_layer.o softmax_layer.o data.o matrix.o network.o connected_layer.o cost_layer.o parser.o option_list.o darknet.o detection_layer.o captcha.o route_layer.o writing.o box.o nightmare.o normalization_layer.o avgpool_layer.o coco.o dice.o yolo.o detector.o layer.o compare.o classifier.o local_layer.o swag.o shortcut_layer.o activation_layer.o rnn_layer.o gru_layer.o rnn.o rnn_vid.o crnn_layer.o demo.o tag.o cifar.o go.o batchnorm_layer.o art.o region_layer.o reorg_layer.o reorg_old_layer.o super.o voxel.o tree.o yolo_layer.o gaussian_yolo_layer.o upsample_layer.o lstm_layer.o conv_lstm_layer.o scale_channels_layer.o sam_layer.o
ifeq ($(GPU), 1)
LDFLAGS+= -lstdc++
OBJ+=convolutional_kernels.o activation_kernels.o im2col_kernels.o col2im_kernels.o blas_kernels.o crop_layer_kernels.o dropout_layer_kernels.o maxpool_layer_kernels.o network_kernels.o avgpool_layer_kernels.o
endif

OBJS = $(addprefix $(OBJDIR), $(OBJ))
DEPS = $(wildcard src/*.h) Makefile include/darknet.h

all: $(OBJDIR) backup results setchmod $(EXEC) $(LIBNAMESO) $(APPNAMESO)

ifeq ($(LIBSO), 1)
CFLAGS+= -fPIC

$(LIBNAMESO): $(OBJDIR) $(OBJS) include/yolo_v2_class.hpp src/yolo_v2_class.cpp
	$(CPP) -shared -std=c++11 -fvisibility=hidden -DLIB_EXPORTS $(COMMON) $(CFLAGS) $(OBJS) src/yolo_v2_class.cpp -o $@ $(LDFLAGS)

$(APPNAMESO): $(LIBNAMESO) include/yolo_v2_class.hpp src/yolo_console_dll.cpp
	$(CPP) -std=c++11 $(COMMON) $(CFLAGS) -o $@ src/yolo_console_dll.cpp $(LDFLAGS) -L ./ -l:$(LIBNAMESO)
endif

$(EXEC): $(OBJS)
	$(CPP) -std=c++11 $(COMMON) $(CFLAGS) $^ -o $@ $(LDFLAGS)

$(OBJDIR)%.o: %.c $(DEPS)
	$(CC) $(COMMON) $(CFLAGS) -c $< -o $@

$(OBJDIR)%.o: %.cpp $(DEPS)
	$(CPP) -std=c++11 $(COMMON) $(CFLAGS) -c $< -o $@

$(OBJDIR)%.o: %.cu $(DEPS)
	$(NVCC) $(ARCH) $(COMMON) --compiler-options "$(CFLAGS)" -c $< -o $@

$(OBJDIR):
	mkdir -p $(OBJDIR)
backup:
	mkdir -p backup
results:
	mkdir -p results
setchmod:
	chmod +x *.sh

.PHONY: clean

clean:
	rm -rf $(OBJS) $(EXEC) $(LIBNAMESO) $(APPNAMESO)

In [ ]:
%cd darknet/
!make

In [ ]:
%cd /content/darknet/
%ls

In [ ]:
!wget https://github.com/AlexeyAB/darknet/releases/download/darknet_yolo_v3_optimal/yolov4.conv.137

In [ ]:
!ls "/content/drive/MyDrive/coding/computer-vision-lab/yolo-training-colab/desarrollo-1/images/datasets"

In [ ]:
%pwd

In [ ]:
!cp -r "/content/drive/MyDrive/coding/computer-vision-lab/yolo-training-colab/desarrollo-1/images/datasets/train" "/content/darknet"
!cp -r "/content/drive/MyDrive/coding/computer-vision-lab/yolo-training-colab/desarrollo-1/images/datasets/test" "/content/darknet"
!cp -r "/content/drive/MyDrive/coding/computer-vision-lab/yolo-training-colab/desarrollo-1/images/datasets/valid" "/content/darknet"

In [ ]:
%ls test/

In [ ]:
%cp train/_darknet.labels data/obj.names

In [ ]:
%mkdir data/obj

%cp train/*.jpg data/obj
%cp valid/*.jpg data/obj
%cp train/*.txt data/obj
%cp valid/*.txt data/obj

In [ ]:
with open("data/obj.data",'w') as plainFile:
  plainFile.write('classes = 1\n');
  plainFile.write('train = data/train.txt\n');
  plainFile.write('valid = data/valid.txt\n');
  plainFile.write('names = data/obj.names\n');
  plainFile.write('backup = backup/');

%cat data/obj.data

In [ ]:
import os

with open('data/train.txt','w') as plainFile:
  for imgName in [f for f in os.listdir('train') if f.endswith('.jpg')]:
    plainFile.write(f'data/obj/{imgName}\n')

with open('data/valid.txt','w') as plainFile:
  for imgName in [f for f in os.listdir('valid') if f.endswith('.jpg')]:
    plainFile.write(f'data/obj/{imgName}\n')

%cat data/valid.txt;

In [ ]:
%pwd
%cat cfg/yolov4-custom2.cfg

In [ ]:
# counting number of classes from the .labels file and declaring batches
# contando el número de clases dentro del archivo .label y declarando los batches

num_classes = sum(1 for line in open('train/_darknet.labels'));
print('number of classes', num_classes)
max_batches = num_classes*2000
stepPoint1 = max_batches * 0.8
stepPoint2 = max_batches * 0.9

if os.path.exists('./cfg/custom-yolov4-detector.cfg'): os.remove('./cfg/custom-yolov4-detector.cfg')

# create and write the .cfg
with open('./cfg/custom-yolov4-detector.cfg','a') as configFile:
  # [net] section
  configFile.write('[net]\n')
  configFile.write('batch=64\n')
  configFile.write('subdivisions=24\n')
  configFile.write('width=416\n')
  configFile.write('height=416\n')
  configFile.write('channels=3\n')
  # contrastive
  # data augmentation
  configFile.write('angle=0\n')
  configFile.write('saturation = 1.5\n')
  configFile.write('exposure = 1.5\n')
  configFile.write('hue = .1\n')
  # optimizator
  configFile.write('momentum=0.949\n')
  configFile.write('decay=0.0005\n')
  configFile.write('learning_rate=0.001\n')
  configFile.write('burn_in=1000\n')
  configFile.write(f'max_batches={max_batches}\n')
  configFile.write('policy=steps\n')
  configFile.write(f'steps={int(stepPoint1)},{int(stepPoint2)}\n')

  with open('cfg/yolov4-custom2.cfg','r') as f:
    content = f2.readlines()
    for line in content:
      f.write(line)
    

print(">> File edited")
%cat ./cfg/custom-yolov4-detector.cfg
  
